# <u>Developing a Bot for Tic Tac Toe on Discord</u>

### Steps:
__1. Explore tic tac toe__
> 1. <u>Representing the board</u>
>> 1. Drawing the board.
>> 2. Letting the player move.
> 2. Branching factor
> 2. Optimal solution


**2. User interface for debugging.**


**3. Text based user interface (conversion to discord bot).**

-------------------------------------------------------------------------------------------------------------------------------

## Exploring Tic Tac Toe

#### Representing the board.
> Basically just using a two dimensional array and iterating over it (bit of fiddling to try and get the board to draw properly).
>> *Side note: this is waaaaay easier to do with a dictionary so that you can access each individual tile directly, but whatever, see code below for an example.*

In [1]:
import random as rnd

global running
global moves_remaining

running = True
moves_remaining = 9

global p1win
global p2win
global draw

p1win = False
p2win = False
draw = False

global player_moving
player_moving = 1

global BOARD_SIZE
BOARD_SIZE = 3

char_x = 'X'
char_o = 'O'

global board
board = [[' ', ' ', ' ',],
         [' ', ' ', ' ',],
         [' ', ' ', ' ',]]

global mboard
mboard = [['1', '2', '3',],
          ['4', '5', '6',],
          ['7', '8', '9',]]

# This implementation is far more consumable than using a two dimensional array.

#board = {'1': ' ', '2': ' ', '3': ' ',
#         '4': ' ', '5': ' ', '6': ' ',
#         '7': ' ', '8': ' ', '9': ' ',}    

Adding some utility methods to help with debugging and development.

In [2]:
def genRandomBoard():
    global board
    rint = 0
    for x in range(len(board)):
        for y in range(len(board[x])):
            if(rint == 1):
                board[x][y] = 'X'
            else:
                board[x][y] = 'O'
            rint = rnd.randrange(1,3,1)

#### Drawing the board

In [3]:
def printBoard(b):
    for x in range(len(b)):
        print('\n- + - + - ')
        for y in range(len(b[x])):
            if(y < 2):
                print(b[x][y] + ' | ', sep=' ', end='', flush=True)
            else:
                print(b[x][y], sep=' ', end='', flush=True)
                
    print('\n- + - + - ')
    
def printGameBoard():
    printBoard(board)

printGameBoard()


- + - + - 
  |   |  
- + - + - 
  |   |  
- + - + - 
  |   |  
- + - + - 


#### Determining whether a player has won.
> This kinda sucked.


> **Columns**
>> Iterate down each column for a given X. If we find a character which does not belong to the corresponding character of our player (X or O), we break. Otherwise, if we reach the size of the board (3-1 because our array index starts at *0*) we can assume that the current player has won the game because they have three of their characters in the column.


> **Rows**
>> This is essentially the same as for columns, but instead we now go along each row a given Y.


> **Diagonal**
>> This is fairly self explanatory. First we check that x = y, which means the move was placed somewhere on the diagonal. Then, we iterate along the diagonal ([1,1], [2,2], [3,3]) and perform the same process as before.


> **Anti-diagonal**
>> This looks more complicated than it is. <br>
>> Imagine the following grid, I have indicated the co-ordinate positions of the cells which we want to evaluate: <br>
>><center> --- | --- | 0,2 <br>
>> --- | 1,1 | --- <br>
>>2,0 | --- | ---  </center>

>> Essentially, we want to make sure that the sum of our co-ordinates for the given move = 2.
>> <br>That way, the move will always have been on the anti-diagonal cells.
>> <br>Using the comparison **(x+y) == (board_size - 1)** we can make sure this is the case.

>> Then, we iterate over the anti-diagonal using **board[ i ][ (board_size - 1) - i ]** <br>
>> This is the general case, but for our purposes we can simplify this to **board[ i ] [2 - i]** <br>
>> This will ensure that we are always on **[2,0]** **[1,1]** or **[0,2]** - *this is best to quickly dry run on a napkin to see the logic*

In [4]:
def checkWinCondition(x, y, player):
    global BOARD_SIZE
    global board
    global p1win
    global p2win
    global draw
    
    char = ' '
    
    if player == 1:
        char = 'X'
    elif player == 2:
        char = 'O'
    
    # Check columns
    for i in range(BOARD_SIZE):
        if board[x][i] != char:
            break
        if i == BOARD_SIZE-1:
            if player == 1:
                p1win = True
            else:
                p2win = True
    
    # Check rows
    for i in range(BOARD_SIZE):
        if board[i][y] != char:
            break
        if i == BOARD_SIZE-1:
            if player == 1:
                p1win = True
            else:
                p2win = True
                
    # Check diag
    if x == y:
        # we are on the diagonal
        for i in range(BOARD_SIZE):
            if board[i][i] != char:
                break
            if i == BOARD_SIZE-1:
                if player == 1:
                    p1win = True
                else:
                    p2win = True
    
    # Check anti-diag
    if (x+y) == (BOARD_SIZE - 1):
        for i in range(BOARD_SIZE):
            if board[i][(BOARD_SIZE-1)-i] != char:
                break
            if i == BOARD_SIZE-1:
                if player == 1:
                    p1win = True
                else:
                    p2win = True
                    
    # Check draw
    if moves_remaining == 0:
        draw = True
            
            

#### Adding player movement
> Essentially, the player selects which square on the board to make a move. \
Each position on the board is labelled from 1 - 9 starting from the top left corner and moving across before moving down, like so:

><center>1 | 2 | 3 <br>
>4 | 5 | 6 <br>
>7 | 8 | 9 </center>

> The player then select which of the numbers to place their move. The characters are automatically alternated between the players.



In [5]:
def printMoves():
    # Display the movement board.
    printBoard(mboard)
    
def move(pos, char):
    global player_moving, board, moves_remaining
    moves_remaining = 1
    for x in range(len(mboard)):
        for y in range(len(mboard[x])):
            if str(pos) == mboard[x][y]:
                print('Pos x: ' + str(x) + ' pos y: ' + str(y))
                board[x][y] = char
                if moves_remaining < 6:
                    checkWinCondition(x, y, player_moving)
    if player_moving == 1:
        player_moving = 2
    elif player_moving == 2:
        player_moving = 1
    
def userInput():
    global player_moving
    print('\nPlayer ' + str(player_moving) + ' turn. \n')
    print('\nThe movements on the board are represented as such:')
    printMoves()
    print('\n\nSelect a position to place your move: ')
    i = input()
    if player_moving == 1:
        move(i, 'X')
    elif player_moving == 2:
        move(i, 'O')

#### Adding the game loop and finishing the game

In [6]:
def reset():
    global running, move_remaining, p1win, p2win, draw, player_moving, BOARD_SIZE, board, mboard
    
    running = True

    moves_remaining = 9

    p1win = False
    p2win = False
    draw = False

    # 1 = p1 2 = p2
    player_moving = 1

    BOARD_SIZE = 3

    board = [[' ', ' ', ' ',],
             [' ', ' ', ' ',],
             [' ', ' ', ' ',]]

    mboard = [['1', '2', '3',],
              ['4', '5', '6',],
              ['7', '8', '9',]]
    
def playAgain():
    print('Play again y/n?')
    i = input()
    if i == 'y':
        reset()
        game()
    else:
        return

def game():
    global running, move_remaining, p1win, p2win, draw, player_moving, BOARD_SIZE, board, mboard
    while(running == True):
        printGameBoard()
        userInput()
        if p1win == True:
            print('Player 1 has won.')
            running = False
        elif p2win == True:
            print('Player 2 has won.')
            running = False
        elif draw == True:
            print('The game has ended in a draw.')
            running = False
    playAgain()

In [ ]:
game()


- + - + - 
  |   |  
- + - + - 
  |   |  
- + - + - 
  |   |  
- + - + - 

Player 1 turn. 


The movements on the board are represented as such:

- + - + - 
1 | 2 | 3
- + - + - 
4 | 5 | 6
- + - + - 
7 | 8 | 9
- + - + - 


Select a position to place your move: 
